In [48]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import shutil
import sklearn as sk

In [53]:
source_file = './Food_Production.csv'
destination_folder = './DATA'
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

#shutil.move(source_file, destination_folder)
DATA_PATH = os.path.join(destination_folder, source_file)
df = pd.read_csv(DATA_PATH, header=0).dropna()

df.head()

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),...,Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,4.281357,...,371.076923,482.4,0.945482,1.907692,2.897446,7.60,5.846154,18786.2,14450.92308,7162.104461
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,9.514379,...,3166.760563,2248.4,1.207271,6.267606,0.759631,2.80,3.943662,49576.3,69825.77465,13449.891480
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.3,4.754098,...,347.647059,59.1,0.628415,2.705882,1.202186,0.88,5.176471,2754.2,16201.17647,3762.568306
11,Nuts,-2.1,0.0,2.1,0.0,0.1,0.1,0.0,0.2,3.113821,...,2531.414574,4133.8,0.069919,0.263319,2.107317,12.96,7.936314,229889.8,140777.58730,37380.455280
12,Groundnuts,0.4,0.0,1.4,0.4,0.1,0.1,0.0,2.4,2.437931,...,707.524828,1852.3,0.556897,1.233766,1.570690,9.11,3.479756,61797.9,23605.00382,10654.810340


In [50]:
y = df['Total_emissions']

#dropping the y label and the string features
X = df.drop(columns='Total_emissions')
string_columns = X.select_dtypes(include=['object']).columns
X = X.drop(columns= string_columns)
X.head()

,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),Eutrophying emissions per kilogram (gPO₄eq per kilogram),Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein),...,Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
3,0.0,0.0,1.4,0.0,0.1,0.1,0.0,4.281357,11.23,8.638462,...,371.076923,482.4,0.945482,1.907692,2.897446,7.60,5.846154,18786.2,14450.92308,7162.104461
4,0.0,0.0,3.6,0.1,0.1,0.1,0.1,9.514379,35.07,49.394366,...,3166.760563,2248.4,1.207271,6.267606,0.759631,2.80,3.943662,49576.3,69825.77465,13449.891480
5,0.0,0.0,0.2,0.0,0.1,0.0,0.0,4.754098,3.48,20.470588,...,347.647059,59.1,0.628415,2.705882,1.202186,0.88,5.176471,2754.2,16201.17647,3762.568306
11,-2.1,0.0,2.1,0.0,0.1,0.1,0.0,3.113821,19.15,11.726883,...,2531.414574,4133.8,0.069919,0.263319,2.107317,12.96,7.936314,229889.8,140777.58730,37380.455280
12,0.4,0.0,1.4,0.4,0.1,0.1,0.0,2.437931,14.14,5.401070,...,707.524828,1852.3,0.556897,1.233766,1.570690,9.11,3.479756,61797.9,23605.00382,10654.810340


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.decomposition import PCA

#standardizing the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = np.array(y).reshape(-1, 1)
y_scaled = scaler.fit_transform(y)

#apply PCA
n_components = 5
pca = PCA(n_components=n_components)
X_filtered = pca.fit_transform(X_scaled)

X_filtered = pd.DataFrame(X_filtered)
print(X_filtered.head())

#split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_scaled, test_size = 0.2, random_state = 42)


          0         1         2         3         4
0 -2.204211  1.031812  0.818212 -0.894043  0.166296
1 -1.475005 -0.662449 -0.354523  0.373484  0.380645
2 -2.386802  1.356762  0.942249 -0.800813  0.180271
3 -1.640673 -3.065070 -0.340562  0.240655  0.515614
4 -1.719154  0.105909  0.624815 -0.937923  0.463312


In [52]:
#train the model
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_pred, y_test)
r2_score_model = r2_score(y_pred, y_test)

print(mse)
print(r2_score_model)

0.08887974306817761
0.8763569629153207
